# Kontekstualizovano ugnježdavanje reči

### Uvod

**Kontekstualizovano ugnježdavanje reči (Contextualized Word Embeddings, CWE)** je postupak formiranja novih reprezentacija reči u zavisnosti od konteksta u kome se reči nalaze. Ideja je pridružiti rečima vektorske reprezantcija takve da je uzet u obzir i kontekst. Na taj način se obuhvata upotreba reči u različitim kotekstima i to znanje ugrađuje u vektorsku reprezentaciju. Formalno, reprezentacije reči odnosno tokena su funkcije od ulazne rečenice. Na primer, reč "Vašington" u rečenici "Univerzitet u Vašingtonu" treba biti protunačena kao naziv ustanove, jer se u tom kontekstu i koristi, a ne kao lično ime ili naziv grada.

Pre razvoja CWE tehnika uglavnom se pristupalo pridruživanju jednog globalnog značenja reči među ostalim rečima u velikim korpusima teksta pomoću modela nenadgledanog učenja:
- Joseph Turian, Lev Ratinov, and Yoshua Bengio. 2010. "Word representations: a simple and general method for semi-supervised learning", Koferencija o empirijskim metodama i obradi prirodnih jezika 2014.
- Tomas Mikolov, Kai Chen, Greg Corrado, and Jeffrey Dean. 2013. "Efficient estimation of word representations in vector space"
- Jeffrey Pennington, Richard Socher, and Christopher Manning. 2014. "Glove: Global vectors for word representation", Koferencija o empirijskim metodama i obradi prirodnih jezika 2014.

Ovi radovi nisu uzimali u obzir kontekst u kome se reč nalazi. Jedan od prvih radova koji je uveo kontekst u reporezentacije reči bio je "Semi-supervised sequence tagging with bidirectional language models", Matthew E. Peters, Waleed Ammar, Chandra Bhagavatula, Russell Power 2017. Smatra se pretečom modernih CWE modela, koji danas važe za poslednju reč tehike (state-of-the-art). Među njima su radovi:
- Matthew E Peters, Mark Neumann, Mohit Iyyer, Matt Gardner, Christopher Clark, Kenton Lee, and Luke Zettlemoyer. 2018. "Deep contextualized word repre-sentations", poznatiji kao ELMO model
- Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. 2018. "Bert: Pre-training of deep bidirectional transformers for language understanding"
- Zhilin Yang, Zihang Dai, Yiming Yang, Jaime Carbonell, Ruslan Salakhutdinov, and Quoc V Le. 2019. "Xlnet: Generalized autoregressive pretraining for language understanding"
- Colin Raffel, Noam Shazeer, Adam Roberts, Katherine Lee, Sharan Narang, Michael Matena, Yanqi Zhou, Wei Li, and Peter J Liu. 2019. "Exploring the limits of transfer learning with a unified text-to-text transformer"

Modeli predstavljni u ovim radovima pokazali su da dobijene CWE reprezentacije reči postižu izuzetne performanse na različitim zadacima obrade prirodnih jezika kao što su klasifikacija teksta, odgovaranje na pitanja, rezimiranje teksta.

### Sadržaj projekta
- U uvodnoj (ovoj) svesci biće predstavljen "*Contextual String Embeddings for Sequence Labeling*" Alan Akbik, Duncan Blythe i Roland Vollgraf 2020. i CWE reprezentacije po nazivom "*Contextual string embeddings*" predstavljen u radu. Nučnici su sve svoje modele i kod organizovali u *[Flair](https://github.com/flairNLP/flair)* radni okvir koji je javno dostupan. Biće predstavljen3 i neke od osnovnih funkcionalnosti Flair radnog okvira.

- U drugoj svesci biće predstavljen način za učitavanje skupova podataka koje Flair nudi kao i opšti postupak treniranje modela za prepoznavanje *upos* etiketa uz korišćenje CWE modela iz radnog okvira. Model je treniran na skupu rečenica na srpskom jeziku i rezltati su prikazani na kraju sveske.

- U trećoj svesci predstavljen je postupak formiranja CWE modela nad proizvoljnim korupuson. Korišćeni su tekstovi na srpsom jeziku. Model je dalje iskorišćem za treniranje modela za prepoznavanje *upos* etiketa i rezultati su prikazani na kraju sveske

### "Contextual String Embeddings for Sequence Labeling" Alan Akbik, Duncan Blythe i Roland Vollgraf 2020.

Autori su u radu predstavili model za formiranje CWE reprezentacija reči. Reprezentacije koje model daje su nazvali "*Contextual string embeddings*". Glavna karakteristika ovog jezičkog model je to što je on formiran na nivou pojedinačnih karaktera u rečenici. To znači da se za kotekst jedne reči uzima u obzir kompletna sekvenca karaktera ispred i iza nje. Ova karakteristika izdvaja model od ostalih koji pri formiranju konteksta u kome se reč nalazi uzimaju u obzi samo okolne reči a ne cele sekvence karaktera.

#### Arhitektura modela

Za formiranje "Contextual string embeddings" reprezentacija autori su iskoristili LSTM varijanu rekurentnih neuronskih mreža za koju navode da se pokazala kao bolji pristup pri formiranju jezičkih modela u donsu na n-grame, pozivajući se na rezultate iz ranijih radova. Razlog tome je činjenica da LSTM neuralne mreže mogu fleksibilno da modeluju dugoročne zavisnosti sa svojim skrivenim stanjem. Kao što je već rečeno, osnovna grdivna jedinica modela je karakter što za posledicu ima to da se rečenica tretira kao sekvenca karaktera prosleđena LSTM-u, a sam LSTM je treniran da u svakoj tački te sekvence predviđa sledeći karakter.

Formalno, cilj jezičkog modela zasnovanog na karakterima je da dobro modeluje raspodelu niza karaktera $P(x_{0:T})$, gde je $(x_0, x_1, ..., x_T):=x_{0:T}$. Treniranjem modela uči se uslovna raspodela karaktera $x_t$ pri datom nizu karaktera, odnosno $P(x_t|x_0,...,x_{t-1})$. Tada se raspodela cele rečenice (čitavog niza karaktera) može dekomponovati na prozvod uslovnih raspodela pojedinačnih karaktera pri nizu karaktera koji im prethode, odnosno 

$$P(x_{0:T}) = \prod_{t=0}^{T}{P(x_t|x_{0:t-1})}$$

U LSTM mreži uslovna verovatnoća $P(x_t|x_{0:t-1})$ se aproksimira funkcijom od izlaza iz neuronse mreže $\boldsymbol{h_t}$, odnosno 

$$P(x_t|x_{0:t-1}) \approx \prod_{t=0}^{T}{P(x_t|\boldsymbol{h_t};\theta})$$

$\boldsymbol{h_t}$ predstavlja čitavu prošlost sekvence karaktera. U LSTM-u se računa rekurzivno, uz dodatnu pomoć memorijske ćelije $c_t$

$$\boldsymbol{h_t}(x_{0:t-1}) = f_h(x_{t-1}, \boldsymbol{h_{t-1}}, c_{t-1}; \theta)$$

$$c_t(x_{0:t-1}) = f_c(x_{t-1}, \boldsymbol{h_{t-1}}, c_{t-1}; \theta)$$ 

gde $\theta$ označava parametre modela a $\boldsymbol{h_{-1}}$ i $c_{-1}$ se mogu inicijalizovati na nulu ili tretirati kao deo parametara $\theta$.

Na izlaz iz mreže $\boldsymbol{h_t}$ je postavljen potpuno povezn sloj sa *softmax* aktivacionom funkcijom, tako da je verovatnoća svakog karaktera data sa

$$P(x_t|\boldsymbol{h_t};V)=softmax(V\boldsymbol{h_t} + b)$$

gde su težine $V$ i slobodni članovi $b$ deo parametara modela.

Zbog karakteristike ovakve rekurentne neuronske mreže, da se informaciju u skrivenim stanjima šalju u oba smera, uz gore opisani model unapred može se dobiti i model unazad (*eng. backward model*, odakle i dolazi $b$ u eksponentu u formulama ispod), odnosno

$$P^b(x_t|x_{t+1:T}) \approx \prod_{t=0}^{T}{P^b(x_t|\boldsymbol{h_t^b};\theta})$$

$$\boldsymbol{h_t^b} = f_h^b(x_{t+1}, \boldsymbol{h_{t+1}^b}, c_{t+1}^b; \theta)$$

$$c_t^b = f_c^b(x_{t+1}, \boldsymbol{h_{t+1}}^b, c_{t+1}^b; \theta)$$

Model unapred i model unazad se koriste na sledći način kako bi se formirala "*Contextual string embeddings*" reprezentacija za neku reč:
- Iz modela unapred uzima se izlaz skrivenog sloja nakon poslednjeg karaktera u željenoj reči. Kako je model unapred treniran da predvidi sledći karakter u sekvenci, skriveni sloj do tog trenutka nosi kontekst od početka rečenive sve do odabrane reči, uključujuću i tu reč (smer unapred).
- Slično, iz modela unazad uzima se izlaz skrivenog sloja pre prvog karaktera u željenoj reči. Kako je model unazad treniran da predvidi prethodni karakter u sekvenici, skriveni sloj do tog trenutka nosi kontekst od kraja rečenice sve do odabrane reči, uljučujuči i tu reč (smer unazad).
- Ova dva izlaza se zatim nadovežu kako bi se formirao "*Contextual string embeddings*" za željenu reč koji će nositi informaciju o samoj reči ali i informaciju o okolnom kontekstu

Formalno, za reč čiji su karakteri $t_0, t_1,..., t_n$ "*Contextual string embeddings*" se formira kao
$$w_i^{charLM} := \begin{bmatrix} h^f_{t_{i+1}-1} \\ h^b_{t_{i}-1} \end{bmatrix}$$

Opisani postupak je ilustrovan na slici ispod
![Formiranje CWE](resources/images/cwe_scheme.PNG)

Ovako dobijene "*Contextual string embeddings*" vektorske reprezentacije reči se mogu koristit pri rešavanju različitih problema obrade prirodnih jezika. Način korišćenja je ilustrovan na slici ispod

![Koriscenje CWE](resources/images/cwe_usage.PNG)

### Flair

*Flair* je radni okvir koji su gore pomenuti naučnici razvili. U okviru Flair-a dostupni su svi modeli koji su pomenuti u radu. Modele je moguće učitati već istrenirane i spremne za korišćenje. Moguće je, takođe, trenirati i sopstveni model sa gore pomenutom arhitekturom. Dostupni su i različiti modeli za obradu prodnih jezika npr. modeli za prepoznavanje imenskih entiteta (*eng. named entity recognition, NER*), modeli za klasifikaciju po vrsti reči (*eng.  part-of-speech tagging, PoS*) i drugi. Dostupni su i različiti CWE modeli ali je isto tako moguće trenirati i svoje CWE modele na proizvoljnim korpusima. Radni okvir podržava i automatsko dohvatanje različitih skupova podataka koji su pravljeni za različite namene. Tu su opšti, ručno etiketirani skupovi tekstualnih podataka. Zatim skupovi podataka namenjeni za rešavanje određenih problema obrade prirodnih jezika kao što su pomenuti NER i PoS problemi. Tu su i skupovio sa biomedicinskim podacima koji se koriste za prepoznvanje imenskih entiteta u toj oblasti. Skupovi su dostupni na raličitim jezicima.

Izvorni kod Flair radnog okvira dostupan je na [ovoj](https://github.com/flairNLP/flair) adresi. Za kreiranje modela, Flair u pozadini koristi PyTorch.

U nastavku će biti predstavljene neke od osnovnih funkcionalnosti koje Flair nudi. 

#### Rečenice i tokeni
Postoje dva centralna objekta radnog okvira. To su `Sentence` i `Token` objekti, apstrakcije rečenica i tokena. Rečenica u sebi sadrži sam tekst rečenice i u suštini predstavlja listu tokena.

In [90]:
from flair.data import Sentence

# Formira se objekat recenice
sentence = Sentence('Danas je suncan dan.')

print(sentence)

Sentence: "Danas je suncan dan ."


In [91]:
# Dohvatanje tokena
print(sentence.get_token(4))
print(sentence[3])

Token[3]: "dan"
Token[3]: "dan"


In [92]:
# Iteriranje kroz tokene
for token in sentence:
    print(token)

Token[0]: "Danas"
Token[1]: "je"
Token[2]: "suncan"
Token[3]: "dan"
Token[4]: "."


Kao što se može videti iz primera koda iznad, kada se formira objekat `Sentence` automatski se radi i tokenizacija rečenice. To je moguće kontrolisati zadavanjem `use_tokenizer` argumenta. U tom slučaju se rečenica deli na reči samo po belinama.

In [93]:
untokenized_sentence = Sentence('The grass is green.', use_tokenizer=False)
print(untokenized_sentence)
print(len(untokenized_sentence))

Sentence: "The grass is green."
4


Moguće koristiti i druge tokenizatore

In [94]:
from flair.tokenization import JapaneseTokenizer

tokenizer = JapaneseTokenizer("janome")

japanese_sentence = Sentence("私はベルリンが好き", use_tokenizer=tokenizer)

print(japanese_sentence)

Sentence: "私 は ベルリン が 好き"


A moguće je i incijalno podeliti rečenicu na tokene

In [95]:
sentence = Sentence(['Danas', 'je', 'suncan', 'dan', '.'])
print(sentence)

Sentence: "Danas je suncan dan ."


Ono što Flair takođe nudi jeste **etiketiranje** bilo rečenica bilo tokena. Ova mogućnost će biti dotsa korišćena u nastavku projekta.

In [96]:
sentence = Sentence("Trava je zelena.")

# Postavljanje etikete reci u recenici
# Postavlja se etiketa koja odgovara ner ("Named entity recognition") etiketiranju
sentence[2].set_label('ner', 'color')

# stampanje recenice sa anotacijom
print(sentence)

Sentence: "Trava je zelena ." → ["zelena"/color]


In [97]:
for token in sentence:
    print(token)

Token[0]: "Trava"
Token[1]: "je"
Token[2]: "zelena" → color (1.0)
Token[3]: "."


In [98]:
token = sentence[2]

# Dohvatanje labele
label = token.get_label('ner')

# Stampanje polja koje token sadrzi
print(f'token.text is: "{token.text}"')
print(f'token.idx is: "{token.idx}"')
print(f'label.value is: "{label.value}"')
print(f'label.score is: "{label.score}"')

token.text is: "zelena"
token.idx is: "3"
label.value is: "color"
label.score is: "1.0"


In [99]:
sentence = Sentence('Francuska je trenutni prvak sveta.')

# Moguce je postaviti etikete i celoj recenici 
# Ovo znaci da recenica ima dve 'topic' etikete
sentence.add_label('topic', 'sports')
sentence.add_label('topic', 'soccer')

# Postavlja se i etiketa za jezik
sentence.add_label('language', 'Serbian')

print(sentence)

Sentence: "Francuska je trenutni prvak sveta ." → sports (1.0); soccer (1.0); Serbian (1.0)


In [100]:
# Iteriranje kroz sve etikete
for label in sentence.labels:
    print(label)

Sentence: "Francuska je trenutni prvak sveta ." → sports (1.0)
Sentence: "Francuska je trenutni prvak sveta ." → soccer (1.0)
Sentence: "Francuska je trenutni prvak sveta ." → Serbian (1.0)


In [101]:
# Iteriranje kroz odredjeni skup etiketa
for label in sentence.get_labels('topic'):
    print(label)

Sentence: "Francuska je trenutni prvak sveta ." → sports (1.0)
Sentence: "Francuska je trenutni prvak sveta ." → soccer (1.0)


#### Skupovi podataka

Skupove podataka moguće je učitati iz modula *datasets*. Ukoliko skup nije lokalno sačuvaj Flair će povući skup sa interneta.

In [102]:
import flair.datasets
corpus = flair.datasets.UD_SERBIAN()

2022-09-01 22:26:20,502 Reading data from C:\Users\Goran\.flair\datasets\ud_serbian
2022-09-01 22:26:20,502 Train: C:\Users\Goran\.flair\datasets\ud_serbian\sr_set-ud-train.conllu
2022-09-01 22:26:20,506 Dev: C:\Users\Goran\.flair\datasets\ud_serbian\sr_set-ud-dev.conllu
2022-09-01 22:26:20,506 Test: C:\Users\Goran\.flair\datasets\ud_serbian\sr_set-ud-test.conllu


Skupovi su unapred podeljeni na trening, test i skup za validaciju (dev skup).

In [103]:
# Stampa se broj recenica u trening skupu
print(len(corpus.train))

# Stampa se broj recenica u test skupu
print(len(corpus.test))

# Stampa se broj recenica u validacionom skupu
print(len(corpus.dev))

3328
520
536


In [104]:
# Dohvatanje recenice iz skupa
sentence = corpus.test[0]
print(sentence)

Sentence: "Beograd i Priština postigli dogovor o slobodi kretanja" → ["Beograd"/Beograd/PROPN/Npmsn/nsubj/Nom/Masc/Sing, "i"/i/CCONJ/Cc/cc, "Priština"/Priština/PROPN/Npfsn/conj/Nom/Fem/Sing, "postigli"/postići/VERB/Vmp-pm/root/Masc/Plur/Past/Part/Act, "dogovor"/dogovor/NOUN/Ncmsan/obj/Inan/Acc/Masc/Sing, "o"/o/ADP/Sl/case/Loc, "slobodi"/sloboda/NOUN/Ncfsl/nmod/Loc/Fem/Sing, "kretanja"/kretanje/NOUN/Ncnsg/nmod/Gen/Neut/Sing]


In [105]:
# Stampanje samo PoS oznaka recenice
print(sentence.to_tagged_string('pos'))

Sentence: "Beograd i Priština postigli dogovor o slobodi kretanja" → ["Beograd"/Npmsn, "i"/Cc, "Priština"/Npfsn, "postigli"/Vmp-pm, "dogovor"/Ncmsan, "o"/Sl, "slobodi"/Ncfsl, "kretanja"/Ncnsg]


Flair nudi i automatsko pravljenje **rečnika** iz učitanog korpusa. Moguće je pravite rečnik za sopstveni korpus što će biti urađeno u nastavku projekta.

In [106]:
# Pravljenje rečnika etiketa za upos ("Universal Part-of-Speech") etiketiranje  
upos_dictionary = corpus.make_label_dictionary(label_type='upos')

# Stampanje recnika
print(upos_dictionary)

2022-09-01 22:26:42,664 Computing label dictionary. Progress:



0it [00:00, ?it/s]
1267it [00:00, 12421.39it/s]
3328it [00:00, 12295.18it/s]

2022-09-01 22:26:42,943 Dictionary created for label 'upos' with 18 values: NOUN (seen 18103 times), PUNCT (seen 9351 times), ADJ (seen 8835 times), ADP (seen 7130 times), VERB (seen 6406 times), PROPN (seen 5622 times), AUX (seen 4667 times), DET (seen 2848 times), SCONJ (seen 2713 times), ADV (seen 2543 times), CCONJ (seen 2541 times), PRON (seen 1859 times), NUM (seen 944 times), PART (seen 461 times), X (seen 232 times), INTJ (seen 3 times), SYM (seen 1 times)
Dictionary with 18 tags: <unk>, NOUN, PUNCT, ADJ, ADP, VERB, PROPN, AUX, DET, SCONJ, ADV, CCONJ, PRON, NUM, PART, X, INTJ, SYM


Ukoliko korisniku nije poznato da li se u skupu nalaze etikete neke vrste, moguće je iskoristiti formiranje rečnika nad korpusom sa prosleđenim bilo kakvim argumentom. Tada se štampaju sve vrste etiketa dostupne u skupu ali se i ispaljuje izuzetak kao posledica zadavanja nepostojećeg tipa oznaka koji se može uhvatiti.

In [107]:
try:
    corpus.make_label_dictionary(label_type='abcd')
except Exception as e:
    print("Uhvacena greska")

2022-09-01 22:26:46,091 Computing label dictionary. Progress:



0it [00:00, ?it/s]
3328it [00:00, 30256.80it/s]

2022-09-01 22:26:46,210 ERROR: You specified label_type='abcd' which is not in this dataset!
2022-09-01 22:26:46,212 ERROR: The corpus contains the following label types: 'lemma' (in 3328 sentences), 'upos' (in 3328 sentences), 'pos' (in 3328 sentences), 'dependency' (in 3328 sentences), 'number' (in 3328 sentences), 'case' (in 3325 sentences), 'gender' (in 3324 sentences), 'verbform' (in 3272 sentences), 'tense' (in 3259 sentences), 'person' (in 3185 sentences), 'mood' (in 3182 sentences), 'degree' (in 3118 sentences), 'definite' (in 2926 sentences), 'prontype' (in 2354 sentences), 'voice' (in 2240 sentences), 'reflex' (in 1115 sentences), 'animacy' (in 1045 sentences), 'numtype' (in 1017 sentences), 'poss' (in 615 sentences), 'polarity' (in 428 sentences), 'number[psor]' (in 283 sentences), 'gender[psor]' (in 156 sentences), 'foreign' (in 121 sentences)
Uhvacena greska


#### Modeli

Flair nudi korisniku učitavanje različitih, već istreniranih, modela. Dostupni su "*Contextual string embeddings*" kroz moduo `FlairEmbeddings`, zatim druge vrste ugnježdavanja npr.  "*GloVe*" ugnježdavanje koji koristi pristup zasnovan na rečim a ne na karakterima. Taj model je dostupa kroz moduo `WordEmbeddings`. Dostupni su takođe već trenirani modeli za etiketiranje teksta raznim vrstama etiketa kroz moduo `SequenceTagger`. U nastavku će biti demostrirano učitavanje i korišćenje ovih modela. 

Napomena: Flair ove sve već istrenirane modele dohvata sa interneta ukoliko nisu dostupni lokalno.

In [121]:
from flair.models import SequenceTagger
from flair.embeddings import WordEmbeddings, FlairEmbeddings

In [122]:
# Ucitavamo vec istrenirani GloVe model 
glove_embedding = WordEmbeddings('glove')

In [124]:
# Koristimo recenicu na engleskom jeziku jer su modeli trenirani nad engleskim korpusom
sentence = Sentence('The grass is green.')
glove_embedding.embed(sentence)

# Stampamo vektorsku reprezentaciju tokena
print(sentence[1])
print(sentence[1].embedding)

Token[1]: "grass"
tensor([-0.8135,  0.9404, -0.2405, -0.1350,  0.0557,  0.3363,  0.0802, -0.1015,
        -0.5478, -0.3537,  0.0734,  0.2587,  0.1987, -0.1433,  0.2507,  0.4281,
         0.1950,  0.5346,  0.7424,  0.0578, -0.3178,  0.9436,  0.8145, -0.0824,
         0.6166,  0.7284, -0.3262, -1.3641,  0.1232,  0.5373, -0.5123,  0.0246,
         1.0822, -0.2296,  0.6039,  0.5541, -0.9610,  0.4803,  0.0022,  0.5591,
        -0.1637, -0.8468,  0.0741, -0.6216,  0.0260, -0.5162, -0.0525, -0.1418,
        -0.0161, -0.4972, -0.5534, -0.4037,  0.5096,  1.0276, -0.0840, -1.1179,
         0.3226,  0.4928,  0.9488,  0.2040,  0.5388,  0.8397, -0.0689,  0.3136,
         1.0450, -0.2267, -0.0896, -0.6427,  0.6443, -1.1001, -0.0096,  0.2668,
        -0.3230, -0.6065,  0.0479, -0.1664,  0.8571,  0.2335,  0.2539,  1.2546,
         0.5472, -0.1980, -0.7186,  0.2076, -0.2587, -0.3650,  0.0834,  0.6932,
         0.1574,  1.0931,  0.0913, -1.3773, -0.2717,  0.7071,  0.1872, -0.3307,
        -0.2836,  0.10

In [125]:
# Ucitavamo vec istrenirana Flair ugnjeydavanja. To su ranije pomenuti "Contextual string embeddings"
# Ucitana su ugnjezdavanja unapred i unazad u skladu sa teorijskom pricom od ranije
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')

Učitane modele možemo iskoristiti nad rečenicom da bismo dobili vektorske reprezentacije za svaki od njenih tokena.

In [126]:
# Koristimo recenicu na engleskom jeziku jer su modeli trenirani nad engleskim korpusom
sentence = Sentence('The grass is green.')
flair_embedding_forward.embed(sentence)

# Stampamo dobijene vektorse reprezentacije tokena
for token in sentence:
    print(token)
    print(token.embedding)
    print("==========================================")

Token[0]: "The"
tensor([-0.0021,  0.0005,  0.0469,  ..., -0.0004, -0.0393,  0.0106])
Token[1]: "grass"
tensor([-0.0006,  0.0047,  0.0248,  ..., -0.0004, -0.0236,  0.0117])
Token[2]: "is"
tensor([ 0.0011, -0.0032,  0.0156,  ..., -0.0061,  0.0112,  0.0100])
Token[3]: "green"
tensor([-0.0034,  0.0003,  0.0256,  ..., -0.0026, -0.0118,  0.0455])
Token[4]: "."
tensor([ 0.0008,  0.0002,  0.1262,  ..., -0.0002,  0.0039,  0.0058])


Ugnjždavanja možemo da kombinujemo uz pomoć modula `StackedEmbeddings`. To odgovara nadovezivanju reprezentacija iz skrivenih slojeva LSTM mreže opisnom u delu o [arhitekturi modela](#Arhitektura-modela).

In [127]:
from flair.embeddings import StackedEmbeddings

Sledećim nadovezivanjem ugnježdavanja unapred i unazan dobijamo baš ono ugnježdavanje opisano u radu. Dakle, sada se pravi "*Contextual string embeddings*" model kao na prvoj slici u delu o [arhitekturi modela](#Arhitektura-modela) :D 

In [128]:
 # "Contextual string embeddings" model
stacked_embeddings = StackedEmbeddings([                                        
                                        flair_embedding_forward,
                                        flair_embedding_backward,
                                       ])

In [129]:
sentence = Sentence('Today is a sunny day.')
stacked_embeddings.embed(sentence)
for token in sentence:
    print(token)
    print(token.embedding)
    print("==========================================")

Token[0]: "Today"
tensor([-1.3909e-03, -3.4892e-05,  3.9613e-02,  ..., -1.8384e-03,
        -1.8453e-01, -1.3610e-01])
Token[1]: "is"
tensor([-6.1987e-03, -3.0133e-03,  9.7788e-02,  ..., -8.2525e-05,
        -1.5145e-02, -3.8790e-02])
Token[2]: "a"
tensor([ 3.4312e-02,  1.0655e-04,  3.8369e-02,  ...,  2.8889e-05,
        -1.5626e-02,  1.6645e-02])
Token[3]: "sunny"
tensor([-0.0009, -0.0039,  0.0238,  ..., -0.0092, -0.0488,  0.0052])
Token[4]: "day"
tensor([-0.0005, -0.0004,  0.0262,  ..., -0.0008,  0.0331,  0.0076])
Token[5]: "."
tensor([ 7.0125e-04,  3.7345e-05,  1.0943e-01,  ...,  4.6394e-04,
        -1.7693e-02,  3.2021e-03])


Možemo učitati i neki od modela za etiketiranje

In [130]:
tagger = SequenceTagger.load('upos')

C:\Users\Goran\anaconda3\envs\ml\lib\site-packages\huggingface_hub\file_download.py:621: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-09-01 22:32:06,741 loading file C:\Users\Goran\.flair\models\upos-english\3489359470b8c3b3c6419514a5f1e27ee827089d6a6b345b4fc2cb5f29b70589.15e4b80e0db9ddfa092bb2a03d56050575455bb50729e3c68617a4aa2f7025ec
2022-09-01 22:32:08,241 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, INTJ, PUNCT, VERB, PRON, NOUN, ADV, DET, ADJ, ADP, NUM, PROPN, CCONJ, PART, AUX, X, SYM, <START>, <STOP>


In [131]:
sentence = Sentence('George Washington went to Washington.')

# Previdjanje UPOS etiketa
tagger.predict(sentence)

print(sentence)

Sentence: "George Washington went to Washington ." → ["George"/PROPN, "Washington"/PROPN, "went"/VERB, "to"/ADP, "Washington"/PROPN, "."/PUNCT]


### Zaključak

Ovim su pokrivene osnove CWE koncepta, osnovne funkcionalnosti Flair radnog okvira kao i teorijske osnove i arhitektura iza modela koje okvir nudi. U nastavku se može očekivati pravljenje rečnika na proizvoljnom korpusu, treniranje "Contextual string embeddings" modela na proizvoljnom korpusu, treniranje modela za etiketiranje uz koršćenje vektorskih reprezentacija. 